In [7]:
import polars as pl
from src.power_curve import get_power_at_wind_velocity

data = pl.read_csv("data_sample.csv").lazy()

data = data.with_columns(
    pl.col("timestamp").str.to_datetime("%Y-%m-%d %H:%M:%S", time_unit="ms")
)
data = data.select((pl.col("timestamp").alias("time"), "wsp_150.0"))

data = data.with_columns(
    power=pl.col("wsp_150.0").map_elements(
        lambda value: get_power_at_wind_velocity(value) * 1e-6, return_dtype=pl.Float64
    )
)

data_eager = data.collect()
print(data_eager)
data_eager.plot("time", "power")

shape: (79_461, 3)
┌─────────────────────┬───────────┬───────────┐
│ time                ┆ wsp_150.0 ┆ power     │
│ ---                 ┆ ---       ┆ ---       │
│ datetime[ms]        ┆ f64       ┆ f64       │
╞═════════════════════╪═══════════╪═══════════╡
│ 2015-01-01 00:00:00 ┆ 11.486133 ┆ 10.150362 │
│ 2015-01-01 01:00:00 ┆ 11.095467 ┆ 9.149487  │
│ 2015-01-01 02:00:00 ┆ 11.17293  ┆ 9.342459  │
│ 2015-01-01 03:00:00 ┆ 11.404918 ┆ 9.93657   │
│ 2015-01-01 04:00:00 ┆ 11.084317 ┆ 9.121931  │
│ …                   ┆ …         ┆ …         │
│ 2024-01-30 19:00:00 ┆ 5.024958  ┆ 0.849879  │
│ 2024-01-30 20:00:00 ┆ 3.885518  ┆ 0.392922  │
│ 2024-01-30 21:00:00 ┆ 3.382503  ┆ 0.259224  │
│ 2024-01-30 22:00:00 ┆ 2.913748  ┆ 0.0       │
│ 2024-01-30 23:00:00 ┆ 1.846643  ┆ 0.0       │
└─────────────────────┴───────────┴───────────┘


:Curve   [time]   (power)

In [34]:
import polars as pl

%matplotlib qt5

import matplotlib.pyplot as plt

price = pl.read_csv(
    "european_wholesale_electricity_price_data_hourly/Denmark.csv"
).lazy()

price = price.select(
    pl.col("Price (EUR/MWhe)").alias("price"),
    time=pl.col("Datetime (Local)").str.to_datetime(
        "%Y-%m-%d %H:%M:%S", time_unit="ms"
    ),
)
price = price.filter(pl.col("time").dt.year() < 2020).drop_nulls()
price = price.set_sorted("time")
# data = data.group_by_dynamic(
#     pl.col("time"),
#     every="1",
#     period="1mo"
# ).agg(pl.col("*").exclude("time").mean())

# data = data.collect()

combined = data.join(price, on="time", how="inner")

combined = combined.with_columns(pl.col("power").mul(pl.col("price")).alias("Income"))


combined_eager = combined.collect()


fig, (ax_0, ax_1, ax_2) = plt.subplots(nrows=3, ncols=1, sharex=True)

# ax_0 = fig.add_subplot(111)
# ax_1 = fig.add_subplot(211)
# ax_2 = fig.add_subplot(311)

ax_0.plot(
    combined_eager.select("time"),
    combined_eager.select("Income"),
    label="Income (€ 1 hour avg.)",
    color="green",
)
ax_1.plot(
    combined_eager.select("time"),
    combined_eager.select("power"),
    label="Power (MW)",
    color="blue",
    linestyle="-",
)
ax_2.plot(
    combined_eager.select("time"),
    combined_eager.select("price"),
    label="Price (€/MWh)",
    color="red",
    linestyle="-",
)

fig.legend()

# (MW * h) * (eur / (Mw * h))


combined_eager.write_csv("price_power_income.csv")

In [33]:
combined_eager

time,wsp_150.0,power,price,Income
datetime[ms],f64,f64,f64,f64
2015-01-01 01:00:00,11.095467,9.149487,18.29,167.344118
2015-01-01 02:00:00,11.17293,9.342459,16.04,149.85305
2015-01-01 03:00:00,11.404918,9.93657,14.6,145.07392
2015-01-01 04:00:00,11.084317,9.121931,14.95,136.372874
2015-01-01 05:00:00,10.595004,7.966423,14.5,115.513134
…,…,…,…,…
2019-12-31 19:00:00,11.872616,11.209836,33.85,379.452944
2019-12-31 20:00:00,11.784342,10.961652,33.54,367.653802
2019-12-31 21:00:00,11.035269,9.001373,33.17,298.575545
